### Load libraries

In [1]:
library(Seurat)
library(stringr)
library(viridis)
library(ggplot2)
library(cowplot)
library(cluster)
library(data.table)
library(foreach)
library(doParallel)
library(proxy)
library(ComplexHeatmap)
library(circlize)
library(igraph)
library(qvalue)
library(dplyr)
library(viridis)
library(VGAM)
library(forcats)
library(grDevices)
library(graphics)
library(RColorBrewer)
library(pheatmap)
library(Cairo)
library(slingshot)

createEmptyDf = function( nrow, ncol, colnames = c() ){
  data.frame( matrix( vector(), nrow, ncol, dimnames = list( c(), colnames ) ) )
}

tableread_fast = function(i, header=TRUE, quote="", sep=","){
  tmp = fread(i, header=header, sep=sep, quote=quote, nThread=32)
  tmp = as.data.frame(tmp)
  return(tmp)
}


Attaching SeuratObject

Loading required package: viridisLite

Loading required package: iterators

Loading required package: parallel


Attaching package: ‘proxy’


The following objects are masked from ‘package:stats’:

    as.dist, dist


The following object is masked from ‘package:base’:

    as.matrix


Loading required package: grid

ComplexHeatmap version 2.11.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))


circlize version 0.4.14
CRA

### Load functions

In [2]:
createEmptyDf = function( nrow, ncol, colnames = c() ){
  data.frame( matrix( vector(), nrow, ncol, dimnames = list( c(), colnames ) ) )
}

In [6]:
#Stacked Violin Plot
#https://divingintogeneticsandgenomics.rbind.io/post/stacked-violin-plot-for-visualizing-single-cell-data-in-seurat/

## remove the x-axis text and tick
## plot.margin to adjust the white space between each plot.
## ... pass any arguments to VlnPlot in Seurat
modify_vlnplot<- function(obj, 
                          feature, 
                          pt.size = 0, 
                          plot.margin = unit(c(-0.75, 0, -0.75, 0), "cm"),
                          ...) {
  p<- VlnPlot(obj, features = feature, pt.size = pt.size, ... )  + 
    xlab("") + ylab(feature) + ggtitle("") + 
    theme(legend.position = "none", 
          plot.title= element_blank(),
          axis.title.x = element_blank(),
          axis.text.x = element_blank(), 
          axis.ticks.x = element_blank(), 
          axis.title.y = element_text(family = "Arial", size = 8, angle = 0, colour = "black", face = "plain"), 
          axis.text.y = element_blank(), 
          plot.margin = plot.margin ) 
  return(p)
}
## extract the max value of the y axis
extract_max<- function(p){
  ymax<- max(ggplot_build(p)$layout$panel_scales_y[[1]]$range$range)
  return(ceiling(ymax))
}
## main function
StackedVlnPlot<- function(obj, features,
                          pt.size = 0, 
                          plot.margin = unit(c(-0.05, 0, -0.05, 0), "cm"),
                          ...) {
  
  plot_list<- purrr::map(features, function(x) modify_vlnplot(obj = obj,feature = x, ...))
  
  # Add back x-axis title to bottom plot. patchwork is going to support this?
  plot_list[[length(plot_list)]]<- plot_list[[length(plot_list)]] +
    theme(axis.text.x=element_text(family = "Arial", size=8, face = "plain"),
          axis.ticks.x = element_line())
  
  
  # change the y-axis tick to only max value 
  ymaxs<- purrr::map_dbl(plot_list, extract_max)
  plot_list<- purrr::map2(plot_list, ymaxs, function(x,y) x + 
                            scale_y_continuous(breaks = c(y)) + 
                            expand_limits(y = y))
  
  p<- patchwork::wrap_plots(plotlist = plot_list, ncol = 1, heights = 1)
  return(p)
}

In [7]:
modify_vlnplot_score <- function(obj, 
                          feature, 
                          pt.size = 0, 
                          plot.margin = unit(c(-0.75, 0, -0.75, 0), "cm"),
                          ...) {
  p<- VlnPlot(obj, features = feature, pt.size = pt.size, ... )  + 
    xlab("") + ylab(feature) + ggtitle("") + 
    theme(legend.position = "none", 
          plot.title= element_blank(),
          axis.title.x = element_blank(),
          axis.text.x =  element_text(family = "Arial", size = 8, angle = 0, colour = "black", face = "plain"), 
          axis.ticks.x = element_blank(), 
          axis.title.y = element_blank(), 
          axis.text.y = element_text(family = "Arial", size = 6, angle = 0, colour = "black", face = "plain"), 
          plot.margin = plot.margin ) 
  return(p)
}

In [68]:
##Plot clones in dimentional reduction plot (DimPlot)
#seu: Seurat object / tmp2: identifier of plotting cells

PlotClones = function(seu, tmp2, name.output, meta.data, clone_plot, reduction){
    #Exctract names for cells in plot
    tmp_raw <- as.vector(row.names(meta.data))
    #Extract colors for plot
    p2 = Seurat::DimPlot(object = seu, reduction = reduction, pt.size = 0.1)
    orig_ident_build = ggplot2::ggplot_build(p2)
    orig_ident_build = orig_ident_build$data[[1]]
    orig_ident_build =  orig_ident_build[order(orig_ident_build$group), ]
    ident.cols = unique(orig_ident_build$colour) # Get a vector of unique colors
    names(ident.cols)=c(0:(max(as.numeric(seu@active.ident)-1)))

    #Extract x- and y- axis information for DimPlot
    xmin <- min(orig_ident_build$x)-0.5
    xmax <- max(orig_ident_build$x)+0.5
    ymin <- min(orig_ident_build$y)-0.5
    ymax <- max(orig_ident_build$y)+0.5
    x_label_min <- floor(xmin/10)*10
    x_label_max <- ceiling(xmax/10)*10
    y_label_min <- floor(ymin/10)*10
    y_label_max <- ceiling(ymax/10)*10

    #Dimplot
    p1 <- DimPlot(object = seu, label = FALSE,
                    cells = tmp2,
                    reduction = reduction,
                    cols = ident.cols,
                    pt.size = 0.1) + 
        NoLegend() +
        scale_x_continuous(limits = c(xmin,xmax), breaks= seq(x_label_min,x_label_max,10)) +
        scale_y_continuous(limits = c(ymin,ymax), breaks= seq(y_label_min,y_label_max,10)) +
        theme(axis.title = element_blank(), 
              axis.text = element_blank(),
             axis.line = element_line(size = 0.2))
    
      save_plot(file = name.output, plot_grid(p1, ncol=1, nrow=1), device="tiff", 
                units="in", dpi = 300, base_width = 3, base_height = 3, limitsize=FALSE)
}

In [18]:
#Pseudotime analysis
#seu.sub: create pseudotime for subset of clusters
#red.use: dimentional reduction for plotting pseudotime
Pseudotime.Min <- function(seu, seu.sub, name, dims, start, dir.name){
  #Create lineages and pseudotime
  lineages = slingshot(seu.sub@reductions$pca@cell.embeddings[,dims], 
                       clusterLabels = seu.sub@active.ident, maxit=1000, start.clus=start)
  pseudotime=slingPseudotime(lineages, na = TRUE)
  
  #calculate pseudotime for each lineage
  for (i in c(1:length(lineages@metadata$lineages))){
    pseudotime[,i]=pseudotime[,i]/max(pseudotime[,i][!is.na(pseudotime[,i])])
  }
  if (length(lineages@metadata$lineages) == 1) {
    pseudotime1 = pseudotime[,1]
  } else {
    pseudotime1 = Matrix::rowMeans(pseudotime, na.rm=TRUE)
  }
  
  #Add to MetaData
  seu2 = AddMetaData(object = seu, metadata = pseudotime1, col.name = name)
    
  return(seu2)
}

### Load data

In [20]:
name.input <- "Aoki_3rd_res0.6scTCRmerged.ABnt.AddSig.rda"
load(name.input)

### Figure 3

In [7]:
###Figure 3B, Protting the result of dimentional reduction

dir.name <- "3B"
file.name <- "Fig3B.umap.tiff"

################################# Processing layer ##########################################
dir.create(dir.name)

#Output twice size of figure, compress after pasting in the powerpoint.
p1 = DimPlot(object = seu, reduction = "umap", label = TRUE, label.size = 8, pt.size = 0.2) +
  theme(axis.title.x = element_text(size=16, family = "Arial"), 
        axis.title.y = element_text(size=16, family = "Arial"), 
        axis.text.x = element_text(size=12, colour = 1, family = "Arial"), 
        axis.text.y = element_text(size = 12, colour = 1, family = "Arial")) +
  theme(panel.border = element_rect(fill = NA, size = 1.5)) 

p1 = p1 + theme(legend.position = 'none')
name.output <- str_c(dir.name, file.name, sep = "/")
save_plot(file = name.output, plot(p1), device="tiff", 
          units="in", dpi = 300, base_width = 4.8, base_height = 4.8, limitsize=FALSE)


Warning message in dir.create(dir.name):
“ '3B' はすでに存在します ”


In [13]:
###Figure 3C, Violin plot for marker genes

dir.name <- "3C"
file.name <- "Fig3C.Vlnplot.Cluster.tiff"
genes <- c("Mki67", "Isg15",
           "Pdcd1", "Havcr2", 
           "Ifng", "Gzmb", "Tnfrsf9", 
           "Tcf7", "Il7r", "Sell")

################################# Processing layer ##########################################
dir.create(dir.name)

ppi <- 600
name.output <- str_c(dir.name, file.name, sep = "/")
tiff(name.output, width=2*ppi, height=2.4*ppi, res=ppi)
p <- StackedVlnPlot(obj = seu, features = genes)
plot(p)
dev.off()

Warning message in dir.create(dir.name):
“ '3C' はすでに存在します ”
Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Scal

png 
  2

In [40]:
#Figure3D, Plot gene scores

dir.input <- "Signatures"
dir.name <- "3D"

################################# Processing layer ##########################################
dir.create(dir.name)
files  <- list.files(dir.input, pattern="Sig.")

#Violin Plot
for(i in files){
  sig_name <- str_remove(i, "Sig.")
  sig_name <- str_remove(sig_name, ".txt")
  feature <- paste(sig_name, "1", sep = "")
  
  file.name2 <- paste(sig_name, "tiff", sep = ".")
  name.output <- str_c(dir.name, file.name2, sep = "/")
  ppi <- 600
  tiff(name.output, width=1.5*ppi, height=1*ppi, res=ppi)
  p <- modify_vlnplot_score(obj = seu, feature = feature, pt.size = 0)
  plot(p)
  dev.off()
}

Warning message in dir.create(dir.name):
“ '3D' はすでに存在します ”


In [69]:
#Figure3E: Plot clones of Oligoclonal / Polyclonal fraction in Dimplot

modules <- c("Oligo", "Poly", "nonOL")
dir.name <- "3E"
#Specify clone table merged with dLN overlap status
name.clone.table <- "clone_within_cluster.dLNOL.csv"

############################# Processing layer ######################################
dir.create(dir.name)

#Call clone table merged with dLN overlap status (Stored in "scTCR_WGCNA_combine" directory)
name.clone.table <- str_c("scTCR_analysis", name.clone.table, sep = "/")
clone_within_cluster <- read.csv(name.clone.table, header = TRUE)

#Call metadata and extract cell names
meta.data <- as.data.frame(seu@meta.data)
tmp_raw <- as.vector(row.names(meta.data))

#Output twice size of figure, compress after pasting in the powerpoint.
for(i in modules){
    #Extract clones in each subset
    clone_plot <- dplyr::filter(clone_within_cluster, OL == i)
    clone_names <- clone_plot$names

    #Extract cell names for Dimplot
    data_sub <- dplyr::filter(meta.data, clone.id %in% clone_names)
    clone.id <- as.vector(row.names(data_sub))
    tmp2 = tmp_raw %in% clone.id
    
    #Output
    name.output <- str_c(i, "clone.plot.tiff", sep = ".")
    name.output <- str_c(dir.name, name.output, sep = "/")
    PlotClones(seu, tmp2, name.output, meta.data, clone_plot, "umap")
}


Warning message in dir.create(dir.name):
“ '3E' はすでに存在します ”


In [80]:
#Figure 3G: Correlation analysis between gene score and clone size
#Input; scTCR_analysis/clone_within_cluster.dLNOL.csv

file.name  <- "clone_within_cluster.dLNOL.csv"
dir.input <- "scTCR_analysis"
dir.name <- "3G"

############################# Processing layer ######################################
dir.create(dir.name)

name.input <- str_c(dir.input, file.name, sep = "/")
d <- read.csv(name.input, header = TRUE)
d_sub <- dplyr::select(d, c("temp_freq", "Cytotoxicity1", "Proliferation1", "Tumor.Prog1", "Tumor.Term1", "OL"))
d_sub <- dplyr::filter(d_sub, temp_freq > 0.001)
d_sub <- dplyr::filter(d_sub, OL %in% c("Poly", "Oligo"))

axis_x <- "temp_freq"
axis_y <- c("Tumor.Prog1", "Tumor.Term1")

#Convert frequency to percentage
d_sub$temp_freq <- d_sub$temp_freq*100

for(i in axis_x){
  for(j in axis_y){

    #Extract data
    d_sub2 <- dplyr::select(d_sub, c("OL", i, j))
    names(d_sub2) <- c("OL", "x_axis", "y_axis")
    
    #Output
    ppi <- 600
    name_out <- str_c("x", i, "y", j, "tiff", sep = ".")
    name_out <- str_c(dir.name, name_out, sep = "/")
    tiff(name_out, width=1.5*ppi, height=1.5*ppi, res=ppi)
    p <- ggplot(d_sub2, aes(x=x_axis, y=y_axis, colour=OL)) +
      scale_colour_manual(values=c(Poly="blue", Oligo="red"))+
      geom_point(size=0.8) +
      scale_x_log10() +
      theme_bw(base_size = 6) +
      theme(
        axis.title.x=element_blank(), axis.title.y=element_blank(),
        axis.text.x = element_text(size=6, family="Arial"),
        axis.text.y = element_text(size=6, family="Arial")) +
      guides(colour=FALSE)
    plot(p)
    dev.off()
      
    #Calculate correlation coefficient
    result.name <- str_c(dir.name, "Genescore.correlation.txt", sep = "/")
    cor_t <- cor.test(log10(d_sub2$x_axis), d_sub2$y_axis, use="pairwise", method="pearson")
    pvalue <- cor_t$p.value
    rvalue <- cor_t$estimate
    cor_abs <- abs(rvalue)
    
    #Record pvalues
    tail <- "tumor repertoire rvalue r_abs pvalue\n"
    cor <- sprintf("%f\t", rvalue)
    cor_abs <- sprintf("%f\t", cor_abs)
    p <- sprintf("%f\t", pvalue)
    freq3 <- paste(i, j, cor, cor_abs, p, tail)
    cat(freq3, file = result.name, append = TRUE)
  }
}

Warning message in dir.create(dir.name):
“ '3G' はすでに存在します ”


### Figure S2

In [96]:
###Figure S2A: Marker Gene Heatmap

#Specify Merker gene list
dir.input <- "MarkerGenePlot"
file.name <- "Aoki_3rdALLmarkers_minpct0.1_Adj_p0.05.txt"

top_count <- 5 #Number of genes per cluster
dir.name <- "S2A"

############################# Processing layer ######################################
dir.create(dir.name)

name.input <- str_c(dir.input, file.name, sep = "/")
seu.markers = read.table(name.input, header = TRUE)

#Extract genes for plotting
top10 = seu.markers %>% group_by(cluster) %>% top_n(top_count, avg_log2FC)
top10 = as.data.frame(top10)
top10  = top10 [!duplicated(top10$gene),]
top10 = top10 %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
top10 = as.data.frame(top10)

#Output heatmap
file.name <- paste(dir.name, "S2A.MarkerHeatmap.tiff", sep = "/")
p <- DoHeatmap(seu, features = top10$gene, disp.min = -2.5, disp.max = 2.5,
               size = 3, angle = 0) +
        theme(legend.position = 'bottom',
             axis.text.y = element_text(size=6, family="Arial"),
             legend.title = element_text(size=8, family="Arial"),
             legend.key.height = unit(0.33, 'cm'),
             legend.text = element_text(size=6, family="Arial")) +
        guides(colour=FALSE)
ggsave(file = file.name, plot = p, device="tiff", units="in", dpi = 300,
       width = 3.3, height = 5.25, limitsize=FALSE)


Warning message in dir.create(dir.name):
“ 'S2A' はすでに存在します ”


In [116]:
###Figure S2B: Scatter plot for marker genes
#Output four times size of figure, compress after pasting in the powerpoint.

tmp = c("Pdcd1", "Havcr2", "Tigit", "Tnfrsf9", 
            "Ifng", "Gzmb", "Klrg1", "Cd69",
            "Ccr7", "Tcf7", "Il7r", "Sell", 
            "Cd38", "Mki67", "Mx1", "Isg15")
dir.name <- "S2B"

############################# Processing layer ######################################
dir.create(dir.name)

file.name = paste(dir.name, "Marker.Expression.tiff", sep='/')
ppi <- 600
p <- FeaturePlot(seu, features = tmp, ncol = 4, order = TRUE,
                reduction = "umap", dims=c(1,2), cols = c("grey", "red"), pt.size = 0.1) *
theme(legend.position = 'none',
      axis.title.x = element_blank(), 
        axis.title.y = element_blank(),
        axis.text.x = element_blank(), 
        axis.text.y = element_blank(),
        plot.title= element_text(size=24, family = "Arial"))
save_plot(file = file.name, plot(p), device="tiff", 
          units="in", dpi = 150, base_width = 11.2, base_height = 12.4, limitsize=FALSE)

Warning message in dir.create(dir.name):
“ 'S2B' はすでに存在します ”


In [22]:
###Figure S2D: Pseudotime Analysis between clusters

dir.name <- "S2D"
start <- "7"
seu.sub <- seu
name <- "pseudotime.all"
red.use <- "umap"

############################# Processing layer ######################################
dir.create(dir.name)

#Define PCs for Pseudotime analysis (Call the result of Jackstraw)
tmp = as.data.frame(seu@reductions$pca@jackstraw@overall.p.values)
tmp1 = tmp[tmp$Score>0.05,1]
dims= c(1:(min(tmp1)-1))

#Perform pseudotime analysis
seu2 <- Pseudotime.Min(seu, seu.sub, name, dims, start, dir.name)

#Plot pseudotime in dimentional reduction
name.output=str_c(dir.name, name, sep='/') %>% str_c("from", start, red.use, "pseudotime.tiff", sep = ".")
ppi <- 600
tiff(name.output, width=3.8*ppi, height=3*ppi, res=ppi)
#Output twice size of figure, compress after pasting in the powerpoint.
FeaturePlot(object = seu2, features = name, cols = c("#800080", "#ffee00"),
            reduction = red.use, pt.size = 0.1) +
  theme(plot.title = element_blank(), 
        axis.title.x = element_text(size=16, family = "Arial"), 
        axis.title.y = element_text(size=16, family = "Arial"), 
        axis.text.x = element_text(size=12, colour = 1, family = "Arial"), 
        axis.text.y = element_text(size = 12, colour = 1, family = "Arial"),
       legend.title = element_text(size=12, family = "Arial"),
       legend.text = element_text(size=12, family = "Arial"),
       legend.key.width = unit(0.5, "cm")) +
  theme(panel.border = element_rect(fill = NA, size = 0.75)) 
dev.off()
    
#Plot Pseudotime in violin plot
name.output=str_c(dir.name, name, sep='/') %>% str_c("from", start, "violin", "pseudotime.tiff", sep = ".") 
ppi <- 600
tiff(name.output, width=1.5*ppi, height=1*ppi, res=ppi)
p <- modify_vlnplot_score(obj = seu2, feature = name, pt.size = 0)
plot(p)
dev.off()

#save seurat object with pseudotime
file.name=str_replace(name.input, ".rda", ".pseudotime.rda")
#save(seu2, file=file.name)

Warning message in dir.create(dir.name):
“'S2D' already exists”


png 
  2

png 
  2

In [119]:
###Figure S3E: Calculate clonality of TCR repertoire per clusters

#Specify clone table merged with dLN overlap status
file.name  <- "clone_within_cluster.dLNOL.csv"
dir.name <- "scTCR_analysis"

############################# Processing layer ######################################
dir.create(dir.name)
name.input <- str_c(dir.name, file.name, sep = "/")
d <- read.csv(name.input, header = TRUE)

clust.array <- c("X0", "X1", "X2", "X3", "X4", "X5", "X6", "X7")

output <- data.frame()

for(i in 1:length(clust.array)){
  clust <- clust.array[i]
  d_sub <- select(d, clust)
  names(d_sub) <- "count"
  d_sub <- filter(d_sub, count > 0)
  d_sub$freq <- d_sub$count / sum(d_sub$count)
  
  ##Calculate median and count clonotypes
  
  l <- length(d_sub$freq)
  count <- sum(d_sub$count)
  
  #Calculate 1-Pielou clonality
  e <- log(d_sub$freq)
  pi <- 1+sum(d_sub$freq * e)/log(length(d_sub$freq))
  
  out <- c(l, count, pi)
  
  output <- rbind(output, out)
}

names(output) <- c("clones", "cells", "Pielou")
output$cluster <- clust.array

name.output <- str_c(dir.name, "S2D.Clonality_inCluster.csv", sep = "/")

write.csv(output, name.output, row.names = F)

Warning message in dir.create(dir.name):
“ 'scTCR_analysis' はすでに存在します ”


In [134]:
###Figure S3F: Extract DEGs between Oligoclonal vs Polyclonal fractions

#Specify clone table merged with dLN overlap status
file.name  <- "clone_within_cluster.dLNOL.csv"
dir.input <- "scTCR_analysis"
dir.name <- "S2E"
clones_array <- c("nonOL", "Oligo", "Poly")
#Target for DEG analysis
Subset1 <- "Oligo"
Subset2 <- "Poly"

############################# Processing layer ######################################
dir.create(dir.name)

#Load clone table merged with dLN overlap status
name.input <- str_c(dir.input, file.name, sep = "/")
d <- read.csv(name.input, header = TRUE)

#Call meta.data and cell names
meta.data <- as.data.frame(seu@meta.data)
tmp_raw <- as.vector(row.names(meta.data))

#Integrate OL status into meta.data
meta.data$class <- "nd"
for(i in clones_array){
    #Extract clones in each subset
    clone_plot <- dplyr::filter(clone_within_cluster, OL == i)
    clone_names <- clone_plot$names

    #Extract cell names belonging to clones
    data_sub <- dplyr::filter(meta.data, clone.id %in% clone_names)
    clone.id <- as.vector(row.names(data_sub))
    tmp2 = tmp_raw %in% clone.id

    #Reflect to meta.data
    meta.data$class[tmp2] <- i
}

#Substitute OL status for "ident"
ident <- seu@active.ident 
names.ident <- names(ident)
ident <- as.factor(meta.data$class)
names(ident) <- names.ident
seu@active.ident <- ident

#DEG extracton

DEG.subset.1 = FindMarkers(seu, ident.1 = Subset1, ident.2 = Subset2,
                           test.use="wilcox", only.pos=TRUE, min.pct=0.1,
                           features.use = NULL, nthreads = 12,
                           adj.p.val.threshold=0.05)
name.output <- str_c(Subset1, "Up.csv", sep = ".")
name.output <- str_c(dir.name, name.output, sep = "/")
write.csv(DEG.subset.1, name.output, row.names = TRUE)

DEG.subset.2 = FindMarkers(seu, ident.1 = Subset2, ident.2 = Subset1,
                           test.use="wilcox", only.pos=TRUE, min.pct=0.1,
                           features.use = NULL, nthreads = 12,
                           adj.p.val.threshold=0.05)
name.output <- str_c(Subset2, "Up.csv", sep = ".")
name.output <- str_c(dir.name, name.output, sep = "/")
write.csv(DEG.subset.2, name.output, row.names = TRUE)

#Create DimPlot
mBC2 <- subset(seu, idents = c("Poly", "Oligo"))
genes <- c("Tnfrsf4", "Fas", "Il7r", "Bcl6", "Xcl1", "Ccr7", "Tcf7",
           "Entpd1", "Havcr2", "Gzmb")
mBC2@meta.data$active.ident <- mBC2@active.ident

p <- DotPlot(mBC2, features = genes, cols = c("blue", "red"),
        dot.min = 0, dot.scale = 4,
        scale.by = "size", scale.min = NA, scale.max = NA, group.by=meta.data$active.ident) +
  theme(axis.title = element_blank(), 
        axis.text.y = element_text(size=8, colour = 1, family = "Arial"),
        axis.text.x = element_text(size=8, colour = 1, family = "Arial", angle = 90)) +
  theme(panel.border = element_rect(fill = NA, size = 0.1),
        legend.key.width = unit(0.2, "cm"),
        legend.key.height = unit(0.2, "cm"),
        legend.text = element_text(size=6, colour = 1, family = "Arial"),
        legend.title = element_text(size=6, colour = 1, family = "Arial"),
        plot.margin = unit(c(0.5, 0, 0, 0), "cm"))
file.name <- str_c(dir.name, "DEG.OligovsPoly.tiff", sep="/")
save_plot(file = file.name, plot(p), device="tiff", 
          units="in", dpi = 600, base_width = 3.5, base_height = 1.6, limitsize=FALSE)

Warning message in dir.create(dir.name):
“ 'S2E' はすでに存在します ”


Formal class 'Seurat' [package "SeuratObject"] with 13 slots
  ..@ assays      :List of 1
  .. ..$ RNA:Formal class 'Assay' [package "SeuratObject"] with 8 slots
  .. .. .. ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:878484] 9 12 14 15 16 17 20 26 28 33 ...
  .. .. .. .. .. ..@ p       : int [1:9114] 0 106 196 279 338 410 491 593 664 769 ...
  .. .. .. .. .. ..@ Dim     : int [1:2] 371 9113
  .. .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. .. ..$ : chr [1:371] "Ada" "Adgre1" "Adgrg3" "Anxa5" ...
  .. .. .. .. .. .. ..$ : chr [1:9113] "mouseSampleTag5_Aoki_5" "mouseSampleTag6_Aoki_21" "mouseSampleTag6_Aoki_49" "mouseSampleTag5_Aoki_62" ...
  .. .. .. .. .. ..@ x       : num [1:878484] 1 94 36 28 51 247 920 109 1 9 ...
  .. .. .. .. .. ..@ factors : list()
  .. .. .. ..@ data         :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:878484] 9 12 14 15 16 17 20 26 28 3